In [ ]:
%use dataframe, kandy

In [ ]:
val txt = """Time:      7  15   30
Distance:  9  40  200
""".reader().readLines()

fun String.toInts() = split(' ').filter { it.isNotBlank() }.map { it.toInt() }
val times = txt[0].removePrefix("Time:").toInts()
val distances = txt[1].removePrefix("Distance:").toInts()

"$times / $distances"


In [ ]:
data class Race(val timeAvailable: Int, val bestDistance: Int)
val racesDF = times.zip(distances).map { (t, d) -> Race(t, d) }.toDataFrame()

In [ ]:
var i = 0
val triesDF = racesDF.add("raceNr") { ++i }.add("buttonTime") { (0..timeAvailable).toList() }.explode("buttonTime")

In [ ]:
val triesResultsDF = triesDF.add("relativeTotalDistance") { 
    buttonTime * (timeAvailable - buttonTime) / bestDistance.toDouble()
}

In [ ]:
val progressDF = triesResultsDF.add("t") { (0..timeAvailable).toList() }.explode("t")

In [ ]:
val distancesDF = progressDF.add("relativeDistance") { (t - buttonTime).coerceAtLeast(0) * buttonTime / bestDistance.toDouble() }

In [ ]:
plot(distancesDF) {
    groupBy(raceNr, buttonTime) {
        line {
            hLine {
                yIntercept.constant(1.0)
                color = Color.BLACK
                type = LineType.DASHED
            }
            x(t)
            y(relativeDistance)
            width = 2.0
            color(relativeTotalDistance) {
                scale = continuousColorHue(direction = WheelDirection.CLOCKWISE)
            }
        }
    }
}

In [ ]:
triesResultsDF.groupBy { it.raceNr }.count { it.relativeTotalDistance > 1.0 }
